```
conda create -n py27 python=2.7 anaconda
source activate py27
conda install -c Quantopian zipline
QUANDL_API_KEY=<API KEY> zipline ingest -b quandl # this takes hours but you only do this once forever
conda install networkx==1.9.1   # conda zipline version doesn't enforce this
conda install -c cvxgrp cvxpy libgcc
pip install git+https://github.com/quantopian/alphalens
jupyter notebook
```

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
from __future__ import print_function

import cvxpy as cvx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

# 1. Define Universe


In [ ]:
from research_tools import run_pipeline, get_symbols, get_pricing
from zipline.pipeline.factors import Returns, AverageDollarVolume
from zipline.pipeline import Pipeline

universe = AverageDollarVolume(window_length=120).top(500)

# a pipeline screen controls what is returned **not** what is calculated; a mask controls what is calculated
p = Pipeline(screen=universe)


In [ ]:
p.show_graph()

We will get this just for one day.

In [ ]:
start_date = '2016-01-05'  # must be a valid trading day
end_date = '2016-01-05'    # must be a valid trading day

In [ ]:
start = time.time()
df = run_pipeline(p, start_date, end_date)
end = time.time()
print('Compute time (s): %f' % (end-start))

In [ ]:
df.shape

In [ ]:
tickers = df.index.get_level_values(1).values.tolist()
tickers[0:10]

# 2. Get Data

In [ ]:
dat = get_pricing(tickers, start_date='2011-01-05', end_date='2016-01-05')

In [ ]:
dat.head()

In [ ]:
rets = dat.pct_change()[1:].fillna(0)
n_pos = len(rets.columns)

In [ ]:
rets.head()

In [ ]:
rets.shape

# 2. Create, Fit, and Test Statistical Risk Model

Fit a model with 20 latent risk factors. Return:
    - factor betas
    - factor covariance matrix
    - idiosyncratic variance matrix


In [ ]:
from sklearn.decomposition import PCA

class RiskModelPCA():
    
    ANN_FACTOR = 252
    
    def __init__(self, num_factors):
        self._num_factors = num_factors
        self.num_stocks_ = None
        self.factor_betas_ = None
        self.factor_returns_ = None
        self.common_returns_ = None
        self.residuals_ = None
        self.factor_cov_matrix_ = None
        self.idio_var_matrix_ = None
        self.explained_variance_ratio_ = None

    def fit(self, returns):
        self.num_stocks_ = len(returns.columns)
        mod = PCA(n_components=self._num_factors, svd_solver='full')
        mod.fit(returns)
        
        self.factor_betas_ = pd.DataFrame(
            data=mod.components_.T,
            index=returns.columns
        )
        
        self.factor_returns_ = pd.DataFrame(
            data=mod.transform(rets),
            index=returns.index
        )
        
        self.explained_variance_ratio_ = mod.explained_variance_ratio_
        
        self.common_returns_ = pd.DataFrame(
            data=np.dot(self.factor_returns_, self.factor_betas_.T),
            index=returns.index
        )
        self.common_returns_.columns = returns.columns
        
        self.residuals_ = (returns - self.common_returns_)
        
        self.factor_cov_matrix_ = np.diag(
            self.factor_returns_.var(axis=0, ddof=1)*RiskModelPCA.ANN_FACTOR
        )
        
        self.idio_var_matrix_ = pd.DataFrame(
            data=np.diag(np.var(self.residuals_))*RiskModelPCA.ANN_FACTOR,
            index=returns.columns
        )
        
        self.idio_var_vector_ = pd.DataFrame(
            data=np.diag(self.idio_var_matrix_.values),
            index=returns.columns
        )
        
        self.idio_var_matrix_.columns = index=returns.columns

    def get_factor_exposures(self, weights):
        F = self.factor_betas_.loc[weights.index]
        return F.T.dot(weights)

    def predict(self, weights):
        """ Calculates expected portfolio risk as sqrt(h'XFX'h + h'Sh).
            This will fail if your portfolio has asset weights not in the risk model"""
        all_assets = pd.DataFrame(
            data=np.repeat(0, self.num_stocks_),
            index=self.factor_betas_.index)
        all_assets.loc[weights.index] = weights
        
            
        h = all_assets
        X = self.factor_betas_
        F = self.factor_cov_matrix_
        S = self.idio_var_matrix_
        
        return np.sqrt(h.T.dot(X).dot(F).dot(X.T).dot(h) + h.T.dot(S).dot(h))[0].values[0]


In [ ]:
rm = RiskModelPCA(20)
rm.fit(rets)

### Plot The % of Variance Explained by Each Factor

You will see that the first factor dominates. The precise defintion of each factor in a latent model is unknown, however we can guess at the likely intepretation. What do you think is the best definition for this first factor?

In [ ]:
plt.bar(np.arange(20), rm.explained_variance_ratio_);
plt.title('% of Variance Explained by Each Factor');

### Plot Common Factor Returns

In [ ]:
rm.factor_returns_.loc[:,0:5].cumsum().plot();

### Predict One Period Forward Portfolio Risk

In [ ]:
sample_portfolio = [tickers[i] for i in np.random.choice(len(tickers), 4).tolist()]
sample_portfolio

In [ ]:
weights = pd.DataFrame(data=[0.25,0.25,0.25,0.25], index=sample_portfolio)

In [ ]:
rm.predict(weights)

# 3. Test Risk Model Prediction

In [ ]:
# Pick random 20-day period
# Predict risk for each of 20 days; cumulate the predicted variance
# Run Risk-Ratio Statistic from Mahdavan paper p. 13
# np.sqrt(np.pi/2.0)*(1.0/T)*np.sum(s/fs)

# 4. Create Alpha Factor(s)

In [ ]:
from ics_scheme import Sector
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import CustomFactor, SimpleMovingAverage

universe = AverageDollarVolume(window_length=120).top(500)
p = Pipeline(screen=universe)

# This is a momentum factor
# Hypothesis: higher past 12-month (252 days) returns are proportional to future return
factor_1 = (
    Returns(window_length=252, mask=universe).
    demean(groupby=Sector()).
    rank().
    zscore()
)

# This is a mean reversion factor
# Hypothesis: short-term outperformers(underperformers) compared to their sector will revert 
factor_2 = (
    -Returns(window_length=5, mask=universe).
    demean(groupby=Sector()).
    rank().
    zscore()
)

# This is a mean reversion factor
# Hypothesis: short-term outperformers(underperformers) to their sector will revert 
# With the addition that we smooth the factor output
factor_3 = (
    SimpleMovingAverage(inputs=[factor_2], window_length=5).
    rank().
    zscore()
)



class CTO(Returns):
    """
    Computes the overnight return, per hypothesis from
    https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2554010
    """
    window_length=2

    inputs = [USEquityPricing.open, USEquityPricing.close]
    
    # The opens and closes matrix is 2 rows x N assets, with the most recent at the bottom.
    # As such, opens[-1] is the most recent open, and closes[0] is the earlier close
    
    def compute(self, today, assets, out, opens, closes):
        out[:] = (opens[-1] - closes[0]) / closes[0]

class TrailingOvernightReturns(Returns):
    """
    Sum of trailing 1m O/N returns
    """
    window_safe = True
    window_length = 5
    
    inputs = [CTO(mask=universe)]
    
    def compute(self, today, asset_ids, out, cto):
        out[:] = np.nansum(cto, axis=0)        

factor_4 = (
    TrailingOvernightReturns().
    rank().
    zscore()
)

factor_5 = (
    SimpleMovingAverage(inputs=[factor_4], window_length=5).
    rank().
    zscore()
)

        
p.add(factor_1, 'Momentum_1YR')
p.add(factor_2, 'Mean_Reversion_5Day_Sector_Neutral')
p.add(factor_3, 'Mean_Reversion_5Day_Sector_Neutral_Smoothed')
p.add(factor_4, 'Overnight_Sentiment')
p.add(factor_5, 'Overnight_Sentiment_Smoothed')



It is best practice to inspect the DAG to ensure everything looks as you would expect:

In [ ]:
p.show_graph(format='png')

In [ ]:
start_date = '2014-01-06'  # must be a valid trading day
end_date = '2016-01-05'    # must be a valid trading day

In [ ]:
start = time.time()
df = run_pipeline(p, start_date, end_date)
end = time.time()
print('Compute time (s): %f' % (end-start))

In [ ]:
df.shape

In [ ]:
df.head()

### Important: Date Alignment

When pipeline returns with a date of, e.g., `2016-01-07` this includes data that would be known as of before the **market open** on `2016-01-07`. As such, if you ask for `USEP.close.latest` it will return the closing price from the day before and label the date `2016-01-07`. All factor values assume to be run prior to the open on the labeled day with data known before that point in time.

In [ ]:
df = df.dropna()
df.shape

In [ ]:
df.head()

# 5. Evaluate Alpha Factors

In [ ]:
import alphalens as al

In [ ]:
assets = df.index.levels[1].values.tolist()

We need to get pricing data in order to calculate returns and factor returns. As stated above, the date labeled for the pipeline output is using data as of the day before (t-1). The pricing data we are getting uses the close price as of the labeled date (t). What this means in practice, is we will evaluate the alpha factor return as if we calculated the factor before the market open, but executed on it at that day's close. This is called a **delay 1** setting and is conservative.

In [ ]:
pricing = get_pricing(
    assets,
    start_date,
    end_date,
    'close'
)

In [ ]:
pricing.shape

### Format alpha factors and pricing for `alphalens`

In [ ]:
factor_names = df.columns
factor_data = {}

start_time = time.clock()
for factor in factor_names:
    print("Formatting factor data for: " + factor)
    factor_data[factor] = al.utils.get_clean_factor_and_forward_returns(
        factor=df[factor],
        prices=pricing,
        periods=[1]
    )
end_time = time.clock()
print("Time to get arrange factor data: %.2f secs" % (end_time - start_time))


### Get the Factor-Weight Returns for each Alpha

In [ ]:
ls_factor_returns = []

start_time = time.clock()
for i, factor in enumerate(factor_names):
    ls = al.performance.factor_returns(factor_data[factor])
    ls.columns = [factor]
    ls_factor_returns.append(ls)
end_time = time.clock()
print("Time to generate long/short returns: %.2f secs" % (end_time - start_time))

df_ls_factor_returns = pd.concat(ls_factor_returns, axis=1)
(1+df_ls_factor_returns).cumprod().plot(title='Factor Returns');

Generally this looks ok... "up and to the right".

## Quantile Analysis

It is not enough to look just at the factor weighted return. A good alpha is also monotonic in quantiles. 

In [ ]:
qr_factor_returns = []

start_time = time.clock()
for i, factor in enumerate(factor_names):
    qr = al.performance.mean_return_by_quantile(factor_data[factor])
    qr[0].columns = [factor]
    qr_factor_returns.append(qr[0])
    #print(qr)
end_time = time.clock()
print("Time to generate quantile returns: %.2f secs" % (end_time - start_time))

df_qr_factor_returns = pd.concat(qr_factor_returns, axis=1)


In [ ]:
(10000*df_qr_factor_returns).plot.bar(
    subplots=True,
    sharey=True,
    layout=(4,2),
    figsize=(14, 14),
    legend=False,
    title='Alphas Comparison: Basis Points Per Day per Quantile'
);


What do you observe?

- None of these alphas are **strictly monotonic**; this should lead you to question why this is? Further research and refinement of the alphas needs to be done. What is it about these alphas that leads to the highest ranking stocks in all alphas execpt MR 5D smoothed to *not* perform the best.
- The majority of the return is coming from the **short side** in all these alphas. The negative return in quintile 1 is very large in all alphas. This could also a cause for concern becuase when you short stocks, you need to locate the short; shorts can be expensive or not available at all.
- If you look at the magnitude of the return spread (i.e., Q1 minus Q5), we are working with daily returns in the 0.03%, i.e., **3 basis points**, neighborhood *before all transaction costs, shorting costs, etc.*. Assuming 252 days in a year, thats 7.56% return annualized. Transaction costs may cut this in half. As such, it should be clear that these alphas can only survive in an institutional setting and that leverage will likely need to be applied in order to acheive an attractive return.

### Calcuate the Sharpe Ratio of the Alphas

Generally, a Sharpe Ratio of near 1.0 or higher is an acceptable single alpha for this universe.

In [ ]:
print("Sharpe Ratio (Annualized; Entire Period)")
pd.DataFrame(data=16*df_ls_factor_returns.mean()/df_ls_factor_returns.std(), columns=['Sharpe Ratio']).round(2)

### Turnover Analysis

Without doing a full and formal backtest, we can analyze how stable the alphas are over time. Stability in this sense means that from period to period, the alpha ranks do not change much. Since trading is costly, we always perfer, all other things being equal, that the ranks do not change significantly per period. We can measure this with the **factor rank autocorrelation (FRA)**.

In [ ]:
ls_FRA = []

start_time = time.clock()
for i, factor in enumerate(factor_names):
    print("Calculating the FRA for: " + factor)
    ls = al.performance.factor_rank_autocorrelation(factor_data[factor]).to_frame()
    ls.columns = [factor]
    ls_FRA.append(ls)
end_time = time.clock()
print("Time to generate FRAs: %.2f secs" % (end_time - start_time))
df_ls_FRA = pd.concat(ls_FRA, axis=1)

In [ ]:
df_ls_FRA.plot(title="Factor Rank Autocorrelation");

### Very Important!

What do you notice about the comparision of the Sharpe Ratios, performance curves, and FRAs for the two mean reversion factors?

Answer: the Sharpe ratios and performance curves are almost identical, but the FRA is much higher for the "Smoothed" factor. This means that the smoothed factor will have much lower trading turnover in practice and is a much preferrable factor. The smoothing gives you a turnover reduction effectively for free.

What do you think would happen if we smooth the momentum factor?

Answer: the FRA is very close to 1.0 meaning the factor ranks are very stable. This makes sense since this factor is the trailing 12-month return; as one day passed, the cumulative 12-month return does not change much. As such, we should not expect any increase in FRA nor any improvement in the factor.

## The Combined Alpha Vector

To use these alphas in a portfolio, we need to combine them somehow so we get a single score per stock. This is a area where machine learning can be very helpful. In this module, however, we will take the simplest approach of combination: simply averaging the scores from each alpha.

In [ ]:
selected_factors = factor_names[[1, 2, 4]]
print(selected_factors)

In [ ]:
df['alpha_vector'] = df[selected_factors].mean(axis=1)

In [ ]:
df.head()

# Putting it All Together


In [ ]:
alphas = df[['alpha_vector']]

Get the alpha vector for a single day.

In [ ]:
alpha_vector = alphas.loc[df.index.get_level_values(0)[-1]]
alpha_vector.head()

# 5. Calculate Optimal Portfolio Constrained by Risk Model

You have an alpha model and a risk model. Generally you want to trade as close as possible to the alpha model but limiting your risk as measured by the risk model. Optimization can help here.

\begin{equation*}
\begin{aligned}
& \underset{h}{\text{maximize}}
& & \alpha^T h + \lambda\|h\|_2\\
& \text{subject to}
& & h^T XFX'h + h'Sh \leq b \\
&&& X^Th \preceq k_{\text{max}} \\
&&& X^Th \succeq k_{\text{min}} \\
&&& h^T\mathbb{1} = 0 \\
&&& \|h\|_1 \leq 1 \\
&&& h \succeq u_{\text{min}} \\
&&& h \preceq u_{\text{max}}, 
\end{aligned}
\end{equation*}


In this formulation, we find the holdings vector $h$ which maxmizes the alpha of the resulting portfolio, subject to a number of constraints. In the objective function, we also have a regularization term that penalizes concentrated portfolios.

The first constraint is that the predicted risk be less than some maximum limit. The second and third constraints are on the maximum and minumum portfolio factor exposures. The fourth constraint is the "market neutral constraint: the sum of the weights must be zero. The fifth costraint is the leverage constraint: the sum of the absolute value of the weights must be less than or equal to 1.0. The last are some minimum and maximum limits on indivudual holdings.


Another common formumation is to take a pre-defined target weighting $h^*$ (e.g., a quantile portfolio), and solve to get as close to that portfolio while respecting portfolio-level constraints.

\begin{equation*}
\begin{aligned}
& \underset{h}{\text{minimize}}
& & \|h - h^*\|_2\\
& \text{subject to}
& & h^T XFX'h + h'Sh \leq b \\
&&& X^Th \preceq k_{\text{max}} \\
&&& X^Th \succeq k_{\text{min}} \\
&&& h^T\mathbb{1} = 0 \\
&&& \|h\|_1 \leq 1 \\
&&& h \succeq u_{\text{min}} \\
&&& h \preceq u_{\text{max}}, 
\end{aligned}
\end{equation*}


In [ ]:
def find_optimal_holdings(
    alpha_vector,
    risk_model,
    risk_cap=0.05,
    factor_max=10.0,
    factor_min=-10.0,
    h_max=0.55,
    h_min=-0.55,
    lambda_reg=0.0,
    obj_max_alpha=True):
    
    # we need to be very careful to align the index of each item!
    try:
        F = rm.factor_betas_.loc[alpha_vector.index]
        X = rm.factor_cov_matrix_
        S = np.diag(rm.idio_var_vector_.loc[alpha_vector.index].values.flatten())
        if np.any(np.isnan(S)):
            raise
    except Exception as e:
        print("Error; likely alphas not in risk model: " + str(e))

    w = cvx.Variable(len(alpha_vector))
    f = F.values.T*w
    
    risk = cvx.quad_form(f, X) + cvx.quad_form(w, S)

    if obj_max_alpha:
        obj = cvx.Maximize(
            alpha_vector.values.flatten()*w - 
            lambda_reg*cvx.norm(w, 2)
        )
    else:
        # (a - a.mean)/sum(abs(a))
        h_star = (alpha_vector.values.flatten()-np.mean(alpha_vector.values.flatten())) \
            /np.sum(np.abs(alpha_vector.values.flatten()))
        obj = cvx.Minimize(cvx.norm(w-h_star, 2))

    constraints = [
        sum(cvx.abs(w)) <= 1.0,
        sum(w) == 0.0,
        w <= h_max,
        w >= h_min,
        risk <= risk_cap+risk_cap,
        F.values.T*w <= factor_max,
        F.values.T*w >= factor_min
    ]
    
    prob = cvx.Problem(obj, constraints)
    prob.solve(verbose=True, max_iters=500)

    optimal_weights = np.asarray(w.value).flatten()

    return pd.DataFrame(data=optimal_weights, index=alpha_vector.index)

In [ ]:
optimal_weights = find_optimal_holdings(
    alpha_vector,
    rm
)

We ran an optimization with no constraints except for the leverge and net constraint. What did the optimizer do?

In [ ]:
optimal_weights.loc[optimal_weights[0].abs()>0.00001]

Yikes. It put all the weight in just two stocks. We can see that these two stocks have the max and min alpha respectively.

In [ ]:
print(alpha_vector.loc[alpha_vector.alpha_vector==alpha_vector.alpha_vector.max()])
print(alpha_vector.loc[alpha_vector.alpha_vector==alpha_vector.alpha_vector.min()])


In [ ]:
rm.get_factor_exposures(optimal_weights).plot.bar(
    title='Portfolio Net Factor Exposures',
    legend=False
);

So in order to enforce diversification, we can do a number of things?

- add a portfolio risk cap constraint
- add a max and min position limit constraint
- add a max and min portfolio factor exposure constraint
- add regularization to the objective function

Question: Can we simply add a constraint that says "position count must be greater than N securities?"
Answer: This is an integer constraint and is not handled by convex optimizers.

So let's try two approaches:

### (1) add a regularization parameter

In [ ]:
optimal_weights_1 = find_optimal_holdings(
    alpha_vector,
    rm,
    lambda_reg=5.0
)

In [ ]:
optimal_weights_1.plot.bar(legend=None, title='Portfolio % Holdings by Stock');
x_axis = plt.axes().get_xaxis()
x_axis.set_visible(False)


Nice. Well diversified. And we can also look at the net portfolio factor exposures:

In [ ]:
rm.get_factor_exposures(optimal_weights_1).plot.bar(
    title='Portfolio Net Factor Exposures',
    legend=False
);

### (2) Add Strict Factor Constraints

In [ ]:
optimal_weights_2 = find_optimal_holdings(
    alpha_vector,
    rm,
    h_max=0.02,
    h_min=-0.02,
    risk_cap=0.0015,
    factor_max=0.015,
    factor_min=-0.015,
    obj_max_alpha=False
)



In [ ]:
optimal_weights_2.plot.bar(legend=None, title='Portfolio % Holdings by Stock');
x_axis = plt.axes().get_xaxis()
x_axis.set_visible(False)

How would you compare and constrast these two approaches?

In [ ]:
rm.get_factor_exposures(optimal_weights_2).plot.bar(
    title='Portfolio Net Factor Exposures',
    legend=False
);